## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
#Question for mentor, what does running as 'Shell command' mean?
#Why doesn't the below code work?
! pip install bayesian-optimization

In [2]:
#Why does the below code work (from google search etc)
#What does % mean?
%pip install bayesian-optimization==1.4.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Had to install lightgbm

pip install lightgbm

SyntaxError: invalid syntax (1743840973.py, line 3)

In [5]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [ ]:
#Had to install catboost
#Did other students had to correct for all these erroneous dependencies/compatibility/lack of correct libaries issues?
pip install catboost

In [ ]:
#Testing this command for my curiosity / also to check versions for dependency issues
pip show numpy

In [ ]:
#HAd to upgrade dask to get the above code to work
pip install --upgrade dask pandas

In [ ]:
print(pd.__version__)

In [ ]:
#Had to upgrade pandas
pip install --upgrade pandas

In [ ]:

pip --version

In [6]:
import os
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv',
 'flight_delays_train.csv.zip']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [7]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [12]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

In [9]:
#Testing new bounds for my curiosity
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 8),
    'b': (4, 12)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [13]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 7.022     | 2.156     | 4.866     |
| 2         | 7.996     | 2.573     | 5.423     |
| 3         | 6.708     | 2.576     | 4.132     |
| 4         | 7.987     | 2.575     | 5.412     |
| 5         | 9.382     | 3.0       | 6.382     |


Great, now let's print the best parameters and the associated maximized target.

In [14]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 6.382251026576782}


9.382251026576782

In [ ]:
#Question for mentor - I still don't understand how the optimizer guesses the next best guest for each iteration
#How is the posterior distribution function formed?
#What do the x and y axis represent in the graphs-charts above?

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [15]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [16]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [17]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [18]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
#Question for mentor - does the y (target) variable always have to be converted into a numpy array (like in the above code)?

In [19]:
#Question, for the below code, what is the difference between using train_df['Month'] vs train_df.Month
hi = train_df['Month'].apply(lambda x: x.split('-')[-1]).astype('int64')
hi

0         8
1         4
2         9
3        11
4        10
         ..
99995     5
99996     1
99997     1
99998     4
99999    11
Name: Month, Length: 100000, dtype: int64

In [21]:
#just testing for myself
df = train_df

In [48]:
#just testing for myself
hi3 = df.groupby(['Month', 'UniqueCarrier']).transform('count')

In [49]:
hi3

,DayofMonth,DayOfWeek,DepTime,Origin,Dest,Distance,dep_delayed_15min
0,789,789,789,789,789,789,789
1,537,537,537,537,537,537,537
2,493,493,493,493,493,493,493
3,567,567,567,567,567,567,567
4,1358,1358,1358,1358,1358,1358,1358
...,...,...,...,...,...,...,...
99995,601,601,601,601,601,601,601
99996,345,345,345,345,345,345,345
99997,502,502,502,502,502,502,502
99998,652,652,652,652,652,652,652


In [24]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [51]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

In [ ]:
#Question - What is the point of creating harmonic features Sin and Cos?

In [ ]:
#Question, in this code from above, what does .transform('count') do?
df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')

In [ ]:
#Question - In this code syntax (extracted from above function):
#  df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
# The resulting colum only extracts out the months that are in [12, 1, 2], as in, the final column doesn't include any other month outside those 3 months?

In [ ]:
#Question - same question as above, except with this below code-syntax
# df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')

Concatenate the training and testing dataframes.


In [52]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [53]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [54]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [62]:
#I'm listing out the new df to see how LabelEncoder() changed the previous df
full_df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,1019,8290,11387,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,136,2247,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103


In [63]:
#I see how the categorical values have new ordinal values values via LabelEncoder()
#Question - how does LabelEncoder() assigns the ordinal values? As in, what is the new 1, 2, 3 (is it by alphabetical order)
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [83]:
#Just trying to discern if all encoded values are assigned via alphabetical order
train_df.UniqueCarrier.unique()

array(['AA', 'US', 'XE', 'OO', 'WN', 'NW', 'DL', 'OH', 'AS', 'UA', 'MQ',
       'CO', 'EV', 'DH', 'YV', 'F9', 'AQ', 'TZ', 'HP', 'B6', 'FL', 'HA'],
      dtype=object)

In [77]:
train_df.UniqueCarrier.nunique()

22

In [73]:
full_df.UniqueCarrier.unique()

array([ 1, 19, 21, 16, 20, 14,  7, 15,  3, 18, 13,  5,  8,  6, 22,  9,  2,
       17, 12,  4, 10, 11,  0])

In [78]:
full_df.UniqueCarrier.nunique()

23

In [ ]:
#Question, why does the encoded column now have an extra value? (0)

In [101]:
full_df

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,1019,8290,11387,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,136,2247,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6,5,2,20,71,132,187,1002,1,0,...,16,1513,224,30958,2558,-0.612907,0.790155,1680,766,413
99996,11,24,6,18,213,159,1515,3635,0,0,...,773,4920,9823,13311,1065,-0.799685,-0.600420,6205,955,1331
99997,1,30,2,16,213,260,438,3689,0,0,...,795,278,9823,15343,1149,-0.690251,-0.723570,6331,1719,1330
99998,1,5,5,7,166,19,761,2685,1,0,...,283,11382,3350,14624,1291,-0.509041,0.860742,4650,71,994


Create a list of the categorical features.

In [55]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [116]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [117]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------


TypeError: cv() got an unexpected keyword argument 'early_stopping_rounds'

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [60]:
lgbBO.max

{}

Review the process at each step by using the '.res[0]' function.

In [61]:
lgbBO.res[0]

IndexError: list index out of range